In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import optuna

In [ ]:
prefixes = ['lgbm', 'catboost']
valids = [pd.read_csv(f"output/{p}_valid.csv") for p in prefixes]
submissions = [pd.read_csv(f"output/{p}.csv") for p in prefixes]

In [ ]:
def objective(trial: optuna.Trial):
    p_lgbm = trial.suggest_uniform('lgbm', 0.0, 1.0)
    p_cat = trial.suggest_uniform('cat', 0.0, 1.0)
    y = valids[0]['rating'].values
    a_lgbm = valids[0].drop('rating', axis=1).values
    a_cat = valids[1].drop('rating', axis=1).values
    a = p_lgbm * a_lgbm + p_cat * a_cat
    b = np.argmax(a, axis=1)

    return f1_score(y, b, average='micro')

study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=10)

In [ ]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

In [ ]:
p_lgbm = 0.677
p_cat = 0.598

a_lgbm = submissions[0].drop(['id', 'rating'], axis=1).values
a_cat = submissions[1].drop(['id', 'rating'], axis=1).values
a = p_lgbm * a_lgbm + p_cat * a_cat

submissions['rating'] = a.argmax(axis=1) + 1
submissions[['id', 'rating']].to_csv('blend_submission.csv', index=False)